<a href="https://colab.research.google.com/github/OscarB1994/Dissertation_project_files-/blob/main/Dissertation_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [ ]:
%%capture
! pip install scikit-optimize

In [ ]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from  torch.utils.data import Dataset

import scipy.io as sio
import matplotlib.image as image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import random
import math
import cv2
import csv
import os

import pickle
from sklearn.model_selection import KFold
from sklearn import datasets, svm, metrics

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix 

from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import  RandomizedSearchCV, GridSearchCV, cross_val_score, StratifiedKFold, cross_validate, train_test_split

from skimage import io, transform
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import load_digits
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram

from pylab import imread,subplot,imshow
from time import time
from tqdm import tqdm
from PIL import Image
from google.colab import drive, files

drive.mount('/content/gdrive')

from matplotlib import rc
from matplotlib import style 
import matplotlib.pyplot as plt
style.use('seaborn-bright')
plt.rcParams['font.family'] = "serif"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def listcheck(list, arg):
  '''Checks for particular elements in list and returns total count '''
  n = 0
  for i in range(len(list)):
    if list[i] == arg:
      n += 1
  return n
def unique(list1):
  ''''Returns unique elements of set/list'''
  list_set = set(list1) 
  unique_list = (list(list_set)) 
  return unique_list
def get_data(directory, names):
    '''Function gather data and assigns corresponding class labels'''
    paths, classes = [], []
    for i, dir_ in enumerate(names):
        for j in os.scandir(directory + dir_):
            if (j.is_file()):
                paths.append(j.path)
                classes.append(i) 
    return paths, classes

# Gathering the data




In [ ]:
label_set = 1
param_set = 7 # 7
def dataset_dir_selection():
  pass
lbl = ('Primary_Labels','Secondary_Labels')
grp_list = ('Single40th', 'AV10', 'AV20', 'AV30','AV40', 'STD_images', 'ACF1', 'ACF5')
directory = f"/content/gdrive/My Drive/Tomography_Images/{lbl[label_set]}/{grp_list[param_set]}/"
prim_dir = "/content/gdrive/My Drive/Tomography_Images/Secondary_Unlabelled/"
prim_names = ["Meas194","Meas196","Meas208", "Meas209", "Meas214", "Meas218", "Meas226","Meas227"]
confusion_matrix_title = ['Sing frame', '10 frame average', '20 frame average',
                          '30 frame average','40 frame average', '40 frame standard deviation',
                          'Auto-correlation: Lag 1', 'Auto-correlation: Lag 5']
if label_set ==0:
  label, names, name = 'Primary Labels', ["Bubble_flow","Plug_flow","Stratified_flow", "Slug_flow", "Annular_flow"], ["Bubble", "Plug", "Stratified", "Slug", "Annular"]
elif label_set ==1:
  label, names, name = 'Secondary Labels', ["Bubble_flow","Plug_flow","Stratified_flow","Slug_flow","Transient_flow"], ["Bubble", "Plug", "Stratified", "Slug", "Transient"]

In [ ]:
print('The nominal class tags and class names are:\n')
for i in enumerate(names):
  print(i)
paths, classes = get_data(directory,
                          names)
data = {
    'path': paths,
    'class': classes
       }
data_df = pd.DataFrame(data,
                       columns=['path', 'class'])
# shuffling
data_df = data_df.sample(frac=1).reset_index(drop=True)
paths, classes = get_data(directory, names)
data = {
    'path': paths,
    'class': classes
       }
noshuf_df = pd.DataFrame(data, columns=['path', 'class'])
prim_paths, prim_classes = get_data(prim_dir, prim_names)
prim_data = {
    'path': prim_paths,
    'class': prim_classes
}
prim_df = pd.DataFrame(prim_data, columns=['path', 'class'])
prim_df = prim_df.sample(frac=1).reset_index(drop=True)
print('Original dataframe')
data_df.head()###################################################### used for fitting, validation and testing. 
class_labels_og = tuple(set(data_df['class'].values.tolist()))
print(f'The class labels are {class_labels_og}\n\n')
print('No shuffle dataframe')
noshuf_df.head()
class_labels_shuf = tuple(set(noshuf_df['class'].values.tolist()))
print(f'The class labels are {class_labels_shuf}\n\n')

print('Prime dataframe')
prim_df.head()
class_labels_prime = tuple(set(prim_df['class'].values.tolist()))
print(f'The class labels are {class_labels_prime}\n\n')

# Helper functions

In [ ]:
import copy 
import timeit

print(f"Found {len(data_df)} images.")
test = data_df.path[1998]
x = Image.open(test)
img = cv2.imread(test)
x
freq_2 = [listcheck(data_df['class'], 0),
          listcheck(data_df['class'], 1),
          listcheck(data_df['class'], 2),
          listcheck(data_df['class'], 3),
          listcheck(data_df['class'], 4)]
          
classes_2 = (name)
classes2 = np.arange(len(classes_2))
plt.bar(classes2, freq_2)
plt.title(label)
plt.xlabel('Flow Type')
plt.ylabel('Frequency')
plt.xticks(classes2, classes_2)
plt.show()
df = pd.read_csv("/content/gdrive/My Drive/Tomography_Images/ERT_files/228-ert.csv", header = None, skiprows=2).iloc[:, 2:]

test = df.iloc[-40::,0].mean()
test2 = df.iloc[-20:,0].mean()
# groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=<object object>, observed=False, dropna=True
print(f'The mean value of the last 40 frames of the first pixel is: {test}')
print(f'The mean value of the last 20 frames of the first pixel is: {test2}')
print(f'The mean of the two reparameterisations is================: {(test2+test)/2:.2f}')
rep_40_df = df.groupby( np.arange(len(df))//40 ).mean()
print(f'Before reparameterization: {df.shape[0]} instances.\n'\
      f'=After 40 frame averaging: {rep_40_df.shape[0]} instances')
print('Pixel-wise correlation comparison between reading of both sensors')
fig, ax = plt.subplots(1,2)
fig.set_figwidth(20)
fig.set_figheight(10)
img1 = rep_40_df.iloc[:,0:316]
ax[0].imshow(img1.corr().fillna(0))
img2 = rep_40_df.iloc[:,316:]
ax[1].imshow(img2.corr().fillna(0))

def Acc_comp():
  correct = 0
  total = 0
  # Why don't we need gradients? What happens if we do include gradients? ---> improves computational speed since grad cal not required for infrence 
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
      
          images = images.to(device)
          labels = labels.to(device)

          outputs = model_gpu(images)
          
          _, predicted = torch.max(outputs.data, 1)
          
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the test images: {(100 * correct / total):.2f}%')

def Val_Acc_comp():
  correct = 0
  total = 0
  with torch.no_grad():
      
      # Iterate over the test set
      for data in valid_loader: 
          images, labels = data
          
          images = images.to(device)
          labels = labels.to(device)

          outputs = model_gpu(images)
          
          # torch.max is an argmax operation
          _, predicted = torch.max(outputs.data, 1)
          
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the validation images: { (100 * correct / total):.2f}%')

import itertools
def plot_confusion_matrix(cm,
                          classes,
                          normalize=False,
                          ticks = True,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix very prettily.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    fig = plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size=18)

    # Specify the tick marks and axis text
    if ticks:
      tick_marks = np.arange(len(classes))
      plt.xticks(tick_marks, classes, rotation=90, fontsize=14)
      plt.yticks(tick_marks, classes, fontsize=14)

    # The data formatting
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Print the text of the matrix, adjusting text colour for display
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 size=14)

    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    plt.tight_layout()
    plt.show()


def validation_stats(valid_loader):
  correct = 0
  total = 0
  labeltot =  torch.tensor([], dtype=torch.long)
  predtot =  torch.tensor([], dtype=torch.long)
  validation_loss = 0

  with torch.no_grad():
      
      for data in valid_loader:
          images, labels = data
    
          images = images.to(device)
          labels = labels.to(device)
          outputs = model_gpu(images)
          
          outputted = model_gpu.forward(images)
          batch_loss = criterion(outputted, labels)
          validation_loss += batch_loss.item()

          _, predicted = torch.max(outputs.data, 1)
          
          labeltot = torch.cat((labeltot, labels.cpu()))
          predtot = torch.cat((predtot, predicted.cpu()))

          
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  return labeltot, predtot, (100 * correct / total), (validation_loss / len(valid_loader))


def test_stats(model_gpu):
  correct = 0
  total = 0
  labeltot =  torch.tensor([], dtype=torch.long)
  predtot =  torch.tensor([], dtype=torch.long)
  test_loss = 0

  with torch.no_grad():

      for data in test_loader:
          images, labels = data
    
          images = images.to(device)
          labels = labels.to(device)
          outputs = model_gpu(images)
          
          outputted = model_gpu.forward(images)
          batch_loss = criterion(outputted, labels)
          test_loss += batch_loss.item()

          _, predicted = torch.max(outputs.data, 1)
          
          labeltot = torch.cat((labeltot, labels.cpu()))
          predtot = torch.cat((predtot, predicted.cpu()))

          
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  print(f'Test accuracy: {100*(correct / total):.3f}%')
  print(f'Test Loss: {test_loss / len(test_loader):.3f}%')
  return labeltot, predtot

def validation_stats(valid_loader):
  correct = 0
  total = 0
  labeltot =  torch.tensor([], dtype=torch.long)
  predtot =  torch.tensor([], dtype=torch.long)
  validation_loss = 0

  with torch.no_grad():
      
      for data in valid_loader:
          images, labels = data
    
          images = images.to(device)
          labels = labels.to(device)
          outputs = model_gpu(images)
          
          outputted = model_gpu.forward(images)
          batch_loss = criterion(outputted, labels)
          validation_loss += batch_loss.item()

          _, predicted = torch.max(outputs.data, 1)
          
          labeltot = torch.cat((labeltot, labels.cpu()))
          predtot = torch.cat((predtot, predicted.cpu()))

          
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  return labeltot, predtot, (100 * correct / total), (validation_loss / len(valid_loader))


# to tensor transformation

def compute_img_mean_std(image_paths):

    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        imgs.append(img)
    imgs = np.stack(imgs, axis=2) ####################################################### its not RBG but gray scale
    print(f'Shape of processes image tensor {imgs.shape}')
    imgs = imgs.astype(np.float32) / 255
    # print(imgs)
    # for i in range(3):
    pixels = imgs[:, :, 0, :].ravel() ###############################################
    means.append(np.mean(pixels))
    stdevs.append(np.std(pixels))

    # means.reverse()  # BGR --> RGB
    # stdevs.reverse()
    print(f"normMean = {means}")
    print(f"normStd = {stdevs}")
    return means, stdevs

In [ ]:
# norm_mean, norm_std = compute_img_mean_std(paths)
#### Secondary
#av10
#norm_mean = [0.5446616] #, 0.5446616, 0.5446616]
#norm_std = [0.42959276] #, 0.42959276, 0.42959276]
# av20 second
#norm_mean = [0.54460377] #, 0.54460377, 0.54460377]
#norm_std = [0.42060852] #, 0.42060852, 0.42060852]
#av40second
norm_mean = [0.54443663]#, 0.54443663, 0.54443663]
norm_std = [0.41369697]#, 0.41369697, 0.41369697]
#single40th
#norm_mean = [0.6505153]# , 0.6505153, 0.6505153]
#norm_std = [0.41578674]#, 0.41578674, 0.41578674]
#### Primary
#AV10
#normMean = [0.5497231]# , 0.5497231, 0.5497231]
#normStd = [0.42839104]#, 0.42839104, 0.42839104]
#AV20
#norm_mean = [0.54939455]#, 0.54939455, 0.54939455]
#norm_std = [0.4195459]#, 0.4195459, 0.4195459]
#acf
#normMean = [0.5187708]#, 0.5187708, 0.5187708]
#normStd = [0.38183716]#, 0.38183716, 0.38183716]
#mean
#norm_mean = [0.5495364]#, 0.5495364, 0.5495364] 
#norm_std = [0.41234276]#, 0.41234276, 0.41234276]
#std
#norm_mean = [0.30008742]#, 0.30008742, 0.30008742]
#norm_std = [0.38573563]# 0.38573563, 0.38573563]
#single40th
#normMean = [0.6202374]# , 0.6202374, 0.6202374]
#normStd = [0.42777553]#, 0.42777553, 0.42777553]

#prim_mean, prim_std = compute_img_mean_std(prim_paths)
prim_mean = [0.534819] #, 0.534819, 0.534819]
prim_std = [0.41308674] #, 0.41308674, 0.41308674]

#K Fold CNN


In [ ]:
data_transform = transforms.Compose([
        transforms.Resize(20),
        transforms.CenterCrop(20),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std),
    ])

class Tomography(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        x = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['class'][index]))

        if self.transform:
            x = self.transform(x)
        return x, y 
data_df

In [ ]:
train_split = 0.70 
valid_split = 0.10
batch_size = 32
train_size = int(len(data_df)*train_split)
valid_size = int(len(data_df)*valid_split)

print(f'Number of training instances  : {train_size}')
print(f'Number of validating instances: {valid_size}')
ins_dataset_train = Tomography(
    df=data_df[:(train_size + valid_size)].reset_index(drop=True),
    transform=data_transform)
ins_dataset_test = Tomography(
    df=data_df[(train_size + valid_size):].reset_index(drop=True),
    transform=data_transform)
instance = 1
print(f'Shape of training instance is         : {ins_dataset_train.__getitem__(instance)[0].shape}')
print(f'Label of the corresponding instance is: {ins_dataset_train.__getitem__(instance)[1]}')
ins_dataset_train.__dict__

In [ ]:
train_loader = torch.utils.data.DataLoader(
    ins_dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory = True)
test_loader = torch.utils.data.DataLoader(
    ins_dataset_test,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=1,
    pin_memory = True)
eval_loader = torch.utils.data.DataLoader(
    ins_dataset_test,
    batch_size=1, 
    shuffle=False,
    num_workers=1,
    pin_memory = True)
for i, data in enumerate(eval_loader, 0):
    images, labels = data
    print(f"Batch {i}\nSize: {len(images)}")
    break 
Yt = ins_dataset_test.df['class'].values.tolist()
eval_loader.dataset.df

## Architecture



In [ ]:
# Convolutional neural network
class ConvNet(nn.Module):
    
    def __init__(self, num_classes=len(names)):
        super(ConvNet, self).__init__()
  
        # Add network layers here
        self.conv1 = nn.Sequential(
            
            nn.Conv2d(in_channels=1,
                      out_channels=16,
                      kernel_size=(3,3)), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2))#,
        self.fc1 = nn.Linear(1296,
                             num_classes) 

    def forward(self, x):
        # Complete the graph
        out = self.conv1(x)
        #out = self.conv2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        #out = self.fc2(out)
        return out
model = ConvNet()
layers = 1
runtime = 10

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_gpu = ConvNet().to(device)
from torch import optim
def weight_reset(model):
    for name, module in model.named_children():
      if('conv' in name):
        torch.nn.init.xavier_uniform_(module[0].weight.data)
      else:
        module.reset_parameters()

##Training K-fold

In [ ]:
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from datetime import datetime 

def train_kfold_epochs(num_epochs):

    global model_gpu
    N_splits = 5
    total_acc, current_fold = 0, 0
    training_losses, validation_losses, validation_accs = [], [], []
    best_model_loss, best_acc_fold, best_loss_fold = 10, 10, 10
    best_model_accuracy,best_accuracy_epoch, best_loss_epoch  = 0, 0, 0
    best_model = []
    confusion_matrix_summed = np.zeros((5,5), dtype=int)
    kfold = KFold(n_splits= N_splits)
####
    for fold, (train_index, test_index) in enumerate(kfold.split(ins_dataset_train)):
        training_losses = []
        validation_losses = []
        weight_reset(model_gpu)
        train = Subset(ins_dataset_train, train_index)###########################
        test = Subset(ins_dataset_train, test_index)
        trainloader = DataLoader(train,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=0,
                                pin_memory=False)#######################
        testloader = DataLoader(test,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=0,
                                pin_memory=False)
        for epoch in range(num_epochs):
          print(f'\nFold number {fold + 1} / {kfold.get_n_splits()} \nEpoch { epoch + 1} / {num_epochs} ')
          running_loss = 0.0
          for i, data in enumerate(trainloader, 0):
                images, labels = data
                # Explicitly specifies that data is to be copied onto the device!
                images = images.to(device)  # <----------- And note it's NOT an in-place operation; original
                labels = labels.to(device)  # <----------- variables still exist on CPU
                optimizer.zero_grad()
                outputs = model_gpu(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 10 == 9:
                  print(f'Epoch / Batch number [{epoch + 1} / {i + 1}]')
                  

          # Validation data
          _, _, validation_accuracy, validation_loss = validation_stats(testloader)
          running_loss = running_loss / len(trainloader)
          training_losses.append(running_loss)
          validation_losses.append(validation_loss) 
          validation_accs.append(validation_accuracy)  

          if best_model_loss > validation_loss:

            best_loss_fold = fold + 1
            best_model_loss = validation_loss
            best_loss_epoch = epoch + 1
            best_model = copy.deepcopy(model_gpu)
            print('Best model copied')


          if best_model_accuracy < validation_accuracy:
            best_acc_fold = fold + 1
            best_model_accuracy = validation_accuracy
            best_accuracy_epoch = epoch + 1

          print("  >>> \n"\
                f"  Epoch: {epoch+1}/{num_epochs}, Convolutional layers : {layers}\n"\
                f"  Train loss: {running_loss:.3f} \n"\
                f"  Validation loss and Accuracy : {validation_loss:.3f} , {validation_accuracy:.3f}% \n""  >>> \n"\
                f"  Best Validation Loss and Accuracy: {best_model_loss:.3f} in epoch {best_loss_epoch} and fold {best_loss_fold}, {best_model_accuracy:.3f}% in epoch {best_accuracy_epoch} and fold {best_acc_fold} \n")

        print(f"  Best Validation Loss and Accuracy in fold: {best_model_loss:.3f} in epoch {best_loss_epoch} for fold {current_fold}, {best_model_accuracy:.3f}% in epoch {best_accuracy_epoch} for fold {current_fold} \n")
        current_fold = fold + 1
        total_acc += validation_accuracy
        plt.plot(training_losses, label='Training loss')
        plt.plot(validation_losses, label='Validation loss')
        plt.title(f'Fit graph for fold: {current_fold}')
        plt.legend(frameon=False)
        plt.show()
#############################################################################################################
        Yp = []
        for x,y in eval_loader:
          img, label_true = x.to(device), y.to(device)
          prediction = model_gpu(img).detach().cpu().numpy().tolist()[0] # using GPU for training routine
          y_p = prediction.index(max(prediction))
          Yp.append(y_p)

        os.chdir('/content/gdrive/My Drive/Tomography_Images/results_info')
        time_stamp = str(datetime.now()).replace('-','_').replace(':', '_').replace(' ', '_').split('.')[0] 

        results = confusion_matrix(Yt,
                                   Yp)
        confusion_matrix_summed += results
        plot_confusion_matrix(results,
                              [m.replace('_', ' ') for m in names],
                              title =f'Confusion matrix: Fold {fold+1}',
                              cmap=plt.cm.Blues)
        print(f'\nThe confusion matrix results for results from fold {fold} are: {results}\n')
        with open(f'confusion_results__fold{fold}_{time_stamp}.pickle', 'wb') as file_handle:
          pickle.dump(results, file_handle)
    print(f'Average fold accuracy: {total_acc / kfold.get_n_splits():.3f}')

    plot_confusion_matrix(confusion_matrix_summed,
                          [m.replace('_', ' ') for m in names],
                          title =f'Epochs: {runtime}. Total confusion matrix: {confusion_matrix_title[param_set]}')
    cls_report = classification_report(Yt,
                                       Yp,
                                       target_names=name)
    
    with open(f'classification_report_CNN_{time_stamp}_{lbl[label_set]}_{grp_list[param_set]}.pickle', 'wb') as file_handle:
      pickle.dump(cls_report,file_handle)

    print(classification_report(Yt,
                                Yp,
                                target_names=name))
    pass
    return

## Initialization


In [ ]:
#train_model_epochs(num_epochs)
from torch import optim
import timeit
runtime = 10
LR = 0.0025
MO = 0.9
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_gpu = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
# Stochastic gradient descent
optimizer = optim.SGD(model_gpu.parameters(),
                      lr=LR,
                      momentum=MO) 
model_gpu = model_gpu.to(device)
cpu_train_time = timeit.timeit("train_kfold_epochs(num_epochs)",
                               setup=f"num_epochs={runtime}",
                               number=1,
                               globals=globals())

print(f"learning rate = {LR}")
print(f"momentum = {MO}")
# test_stats(model_gpu)

# CNN




####CNN Loaders


In [ ]:
from matplotlib import image
from matplotlib import pyplot
from PIL import Image
from numpy import asarray

train_split = 0.70
valid_split = 0.10
batch_size = 32

train_size = int(len(data_df)*train_split) 
valid_size = int(len(data_df)*valid_split)

training = data_df[:train_size]
trainval = data_df[:train_size+valid_size]

# **Gaussian noise enrichment**

In [ ]:
targets = unique(training['class'])
size = []
extend, boundary = {}, {}
for i in targets:
  porp = listcheck(training['class'],i)
  if i == 0:
    lst0 = training.sort_values("class").iloc[0:porp]
    l0 = len(lst0)
  if i == 1:
    lst1 = training.sort_values("class").iloc[l0:l0+porp]
    l1 = len(lst1)
  if i == 2:
    lst2 = training.sort_values("class").iloc[l0+l1:l0+l1+porp]
    l2 = len(lst2)
  if i == 3:
    lst3 = training.sort_values("class").iloc[l0+l1+l2:l0+l1+l2+porp]
    l3 = len(lst3)
  if i == 4:
    lst4 = training.sort_values("class").iloc[l0+l1+l2+l3:l0+l1+l2+l3+porp]

    
  print(f"class {i} represents {listcheck(training['class'],i)} of the training elements")
  size.append(listcheck(training['class'],i))

for i in targets:
  diff = max(size) - listcheck(training['class'],i)
  extend[i] = diff
  print(f"{diff} difference for {i}")
  for j in range(diff):
    pass

# bootstrap sampling for class enrichment ----> i.e. with replacement
tba0 = lst0.sample(extend[0], replace = True)
tba1 = lst1.sample(extend[1], replace = True)
tba2 = lst2.sample(extend[2], replace = True)
tba3 = lst3.sample(extend[3], replace = True)
tba4 = lst4.sample(extend[4], replace = True)

In [ ]:
imag0 = []
mean, variance = 0, 0.03 # Gaussian noise parameters  
for i in tqdm(range(len(tba0)), desc='Data enrichment for class 0...'):
  test = image.imread(tba0.iloc[i][0])
  for j in range(20):
    for k in range(20):
      if test[j][k] == 1:
        pass
      else:
        test[j][k] += abs(test[j][k] + np.random.normal(mean,variance,1))
  imag0.append(test)
imag1 = []
for i in tqdm(range(len(tba1)), desc='Data enrichment for class 1...'):
  test = image.imread(tba1.iloc[i][0])
  for j in range(20):
    for k in range(20):
      if test[j][k] == 1:
        pass
      else:
        test[j][k] += abs(test[j][k] + np.random.normal(mean,variance,1))
  imag1.append(test)
imag2 = []
for i in tqdm(range(len(tba2)), desc='Data enrichment for class 2...'):
  test = image.imread(tba2.iloc[i][0])
  for j in range(20):
    for k in range(20):
      if test[j][k] == 1:
        pass
      else:
        test[j][k] += abs(test[j][k] + np.random.normal(mean,variance,1))
  imag2.append(test)
imag3 = []
for i in tqdm(range(len(tba3)), desc='Data enrichment for class 3...'):
  test = image.imread(tba3.iloc[i][0])
  for j in range(20):
    for k in range(20):
      if test[j][k] == 1:
        pass
      else:
        test[j][k] += abs(test[j][k] + np.random.normal(mean,variance,1))
  imag3.append(test)
imag4 = []
for i in tqdm(range(len(tba4)), desc='Data enrichment for class 4...'):
  test = image.imread(tba4.iloc[i][0])
  for j in range(20):
    for k in range(20):
      if test[j][k] == 1:
        pass
      else:
        test[j][k] += abs(test[j][k] + np.random.normal(mean,variance,1))
  imag4.append(test)

zeros = [0]*len(imag0)
ones = [1]*len(imag1)
twos = [2]*len(imag2)
threes = [3]*len(imag3)
fours = [4]*len(imag4)

x, y = [], []
for i in tqdm(range(train_size), desc=' Loading training data...'):
  x.append(np.asarray(Image.open(data_df['path'][i]))/255)
  y.append(data_df['class'][i])

zip0 = list(zip(imag0,zeros))
class_zero = pd.DataFrame(zip0, columns=['path','class'])
zip1 = list(zip(imag1,ones))
class_one = pd.DataFrame(zip1, columns=['path','class'])
zip2 = list(zip(imag2,twos))
class_two = pd.DataFrame(zip2, columns=['path','class'])
zip3 = list(zip(imag3,threes))
class_three = pd.DataFrame(zip3, columns=['path','class'])
zip4 = list(zip(imag4,fours))
class_four = pd.DataFrame(zip4, columns=['path','class'])

zip10 = list(zip(x,y))
original_train = pd.DataFrame(zip10, columns=['path','class'])

In [ ]:
from torch.utils.data import DataLoader
from  sklearn.model_selection import KFold 
import random
data_transform = transforms.Compose([
        transforms.Resize(20),
        transforms.CenterCrop(20),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std)])

balanced_train = pd.concat([class_zero,
                            class_one,
                            class_two,
                            class_three,
                            class_four,
                            original_train],
                            ignore_index = True)
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
ax.imshow(balanced_train['path'][5200])
X = balanced_train['path'].values.tolist()
Y = balanced_train['class'].values.tolist()

Xe, Ye, Xt, Yt = X[:int( 0.10 * len(X))], Y[:int( 0.10 * len(X))],  X[int( 0.10 * len(X)):], Y[:int( 0.10 * len(X))]

print(f'Total number of examples after enriching data eval: {len(Xe)}')
Xe_tensor = torch.tensor(Xe)
Ye_tensor = torch.tensor(Ye)

Xt_tensor = torch.tensor(Xt)
Yt_tensor = torch.tensor(Yt)

Xe_added_dim, Xt_added_dim = [], [] 
for i in range(len(Xe_tensor)):
  Xe_added_dim.append(Xe_tensor[i].unsqueeze_(0))
  Xt_added_dim.append(Xt_tensor[i].unsqueeze_(0))

Xe = torch.stack(Xe_added_dim, dim=0)
Xe = Xe.type(torch.cuda.FloatTensor)

Xt = torch.stack(Xt_added_dim, dim=0)
Xt = Xt.type(torch.cuda.FloatTensor)

In [ ]:
freq_train = [listcheck(balanced_train['class'], 0),
              listcheck(balanced_train['class'], 1),
              listcheck(balanced_train['class'], 2),
              listcheck(balanced_train['class'], 3),
              listcheck(balanced_train['class'], 4)]
classes_train = (names[0],names[1],names[2],names[3],names[4])
classestrain = np.arange(len(classes_train))
plt.bar(classestrain, freq_train)
plt.title('Primary Labels')
plt.xlabel('Flow Type')
plt.ylabel('Frequency')
plt.xticks(classestrain, classes_train)
plt.show()

In [ ]:
train_split = 0.70 # Defines the ratio of train/valid/test data.
valid_split = 0.10
batch_size = 32
train_size = int(len(Xt)*train_split)
valid_size = int(len(Xt)*valid_split)
class Tomography_bal_train(Dataset):
    def __init__(self,
                 df,
                 transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        x = Image.fromarray(np.uint8(cm.gray(self.df['path'][index])*255))
        y = torch.tensor(int(self.df['class'][index]))
        if self.transform:
            x = self.transform(x)
        return x, y 

## Architecture



In [ ]:
# Convolutional neural network
class ConvNet(nn.Module):
    def __init__(self, num_classes=len(names)):
        super(ConvNet, self).__init__()
        # Add network layers here
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=16,
                      kernel_size=(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(1296, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

def weight_reset(model):
    for name, module in model.named_children():
      if('conv' in name):
        torch.nn.init.xavier_uniform_(module[0].weight.data)
      else:
        module.reset_parameters()
model_gpu = ConvNet()
layers = 1
runtime = 10

### Training


In [ ]:
def train_cnn_epochs(num_epochs):
    """ Copy of function train_model_epochs but explicitly copying data to device 
        during training. 
    """
    global model_gpu
    training_losses = []
    validation_losses = []
    validation_accs = []
    best_model_loss = 10
    best_model_accuracy = 0
    best_model = []
    best_accuracy_epoch = 0
    best_loss_epoch = 0
    for epoch in range(num_epochs):
        # Train data
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            # Explicitly specifies that data is to be copied onto the device!
            images = images.to(device)  # <----------- And note it's NOT an in-place operation; original
            labels = labels.to(device)  # <----------- variables still exist on CPU
            optimizer.zero_grad()
            outputs = model_gpu(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
              print('Epoch / Batch [%d / %d]' %
                      (epoch + 1, i + 1))
        # Validation data
        _, _, validation_accuracy, validation_loss = validation_stats(valid_loader)
        running_loss = running_loss / len(train_loader)
        training_losses.append(running_loss)
        validation_losses.append(validation_loss) 
        validation_accs.append(validation_accuracy)  
        if best_model_loss > validation_loss:
          best_model_loss = validation_loss
          best_loss_epoch = epoch + 1
          best_model = copy.deepcopy(model_gpu)
        if best_model_accuracy < validation_accuracy:
          best_model_accuracy = validation_accuracy
          best_accuracy_epoch = epoch + 1
        print("  >>> \n"
              f"  Epoch: {epoch+1}/{num_epochs}, Convolutional layers : {layers}\n"
              f"  Train loss: {running_loss:.3f} \n"
              f"  Validation loss and Accuracy : {validation_loss:.3f} , {validation_accuracy:.3f}% \n""  >>> \n" 
              f"  Best Validation Loss and Accuracy: {best_model_loss:.3f} in epoch {best_loss_epoch} , {best_model_accuracy:.3f}% in epoch {best_accuracy_epoch} \n"
        )
        model.train()
    plt.plot(training_losses, label='Training loss')
    plt.plot(validation_losses, label='Validation loss')
    plt.legend(frameon=False)
    plt.show()
    model_gpu = best_model.to(device)

##Training K-fold

In [ ]:
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from datetime import datetime 
def train_kfold_epochs(num_epochs):
    global model_gpu
    N_splits = 5
    total_acc, current_fold = 0, 0
    training_losses, validation_losses, validation_accs, best_model = [], [], [], []
    best_model_loss, best_acc_fold, best_loss_fold = 10, 10, 10
    best_model_accuracy,best_accuracy_epoch, best_loss_epoch  = 0, 0, 0
    confusion_matrix_summed = np.zeros((5,5),
                                       dtype=int)
    kfold = KFold(n_splits= N_splits)

    for fold, (train_index, test_index) in enumerate(kfold.split(list(zip(X,Y_tensor)))):
        training_losses = []
        validation_losses = []
        weight_reset(model_gpu)

        data_train = TensorDataset(Xt[train_index], Yt_tensor[train_index])
        data_test = TensorDataset(Xt[test_index], Yt_tensor[test_index])
        data_eval = TensorDataset(Xe, Ye_tensor)

        train_loader = DataLoader(data_train,
                                  batch_size=batch_size,
                                  shuffle=True)
        train_loader = DataLoader(data_test,
                                  batch_size=batch_size,
                                  shuffle=True)
      
        eval_loader = DataLoader(data_eval, 
                                 batch_size=1)
        # trainloader = DataLoader(train,
        #                         batch_size=batch_size,
        #                         shuffle=True,
        #                         num_workers=0,
        #                         pin_memory=False)#######################
        # testloader = DataLoader(test,
        #                         batch_size=batch_size,
        #                         shuffle=True,
        #                         num_workers=0,
        #                         pin_memory=False)
        for epoch in range(num_epochs):
          print(f'\nFold number {fold + 1} / {kfold.get_n_splits()} \nEpoch { epoch + 1} / {num_epochs} ')
          running_loss = 0.0
          for i, data in enumerate(train_loader, 0):
                images, labels = data
                # Explicitly specifies that data is to be copied onto the device!
                images = images.to(device)  # <----------- And note it's NOT an in-place operation; original
                labels = labels.to(device)  # <----------- variables still exist on CPU
                optimizer.zero_grad()
                outputs = model_gpu(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 10 == 9:
                  print(f'Epoch / Batch number [{epoch + 1} / {i + 1}]')

          # Validation data
          _, _, validation_accuracy, validation_loss = validation_stats(test_loader)
          running_loss = running_loss / len(train_loader)
          training_losses.append(running_loss)
          validation_losses.append(validation_loss) 
          validation_accs.append(validation_accuracy)  
          if best_model_loss > validation_loss:
            best_loss_fold = fold + 1
            best_model_loss = validation_loss
            best_loss_epoch = epoch + 1
            best_model = copy.deepcopy(model_gpu)
            print('Best model copied')
          if best_model_accuracy < validation_accuracy:
            best_acc_fold = fold + 1
            best_model_accuracy = validation_accuracy
            best_accuracy_epoch = epoch + 1
          print("  >>> \n"\
                f"  Epoch: {epoch+1}/{num_epochs}, Convolutional layers : {layers}\n"\
                f"  Train loss: {running_loss:.3f} \n"\
                f"  Validation loss and Accuracy : {validation_loss:.3f} , {validation_accuracy:.3f}% \n""  >>> \n"\
                f"  Best Validation Loss and Accuracy: {best_model_loss:.3f} in epoch {best_loss_epoch} and fold {best_loss_fold}, {best_model_accuracy:.3f}% in epoch {best_accuracy_epoch} and fold {best_acc_fold} \n")

        print(f"  Best Validation Loss and Accuracy in fold: {best_model_loss:.3f} in epoch {best_loss_epoch} for fold {current_fold}, {best_model_accuracy:.3f}% in epoch {best_accuracy_epoch} for fold {current_fold} \n")
        current_fold = fold + 1
        total_acc += validation_accuracy
        plt.plot(training_losses, label='Training loss')
        plt.plot(validation_losses, label='Validation loss')
        plt.title(f'Fit graph for fold: {current_fold}')
        plt.legend(frameon=False)
        plt.show()
#############################################################################################################
        Yp = []
        for x,y in eval_loader:
          img, label_true = x.to(device), y.to(device)
          prediction = model_gpu(img).detach().cpu().numpy().tolist()[0] # using GPU for training routine
          y_p = prediction.index(max(prediction))
          Yp.append(y_p)

        os.chdir('/content/gdrive/My Drive/Tomography_Images/results_info')
        time_stamp = str(datetime.now()).replace('-','_').replace(':', '_').replace(' ', '_').split('.')[0] 

        results = confusion_matrix(Yt,
                                   Yp)
        confusion_matrix_summed += results
        plot_confusion_matrix(results,
                              [m.replace('_', ' ') for m in names],
                              title =f'Confusion matrix: Fold {fold+1}',
                              cmap=plt.cm.Blues)
        print(f'\nThe confusion matrix results for results from fold {fold} are: {results}\n')
        with open(f'confusion_results__fold{fold}_{time_stamp}.pickle', 'wb') as file_handle:
          pickle.dump(results, file_handle)
    print(f'Average fold accuracy: {total_acc / kfold.get_n_splits():.3f}')
    plot_confusion_matrix(confusion_matrix_summed,
                          [m.replace('_', ' ') for m in names],
                          title =f'Epochs: {runtime}. Total confusion matrix: {confusion_matrix_title[param_set]}')
    cls_report = classification_report(Yt,
                                       Yp,
                                       target_names=name)
    with open(f'classification_report_CNN_{time_stamp}_{lbl[label_set]}_{grp_list[param_set]}.pickle', 'wb') as file_handle:
      pickle.dump(cls_report,file_handle)
    print(classification_report(Yt,
                                Yp,
                                target_names=name))
    pass
    return

## Initialization

In [ ]:
from torch import optim
LR, MO, runtime = 0.0025, 0.9, 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_gpu = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_gpu.parameters(),
                      lr=LR,
                      momentum=MO)
 
model_gpu = model_gpu.to(device)

cpu_train_time = timeit.timeit(
    "train_kfold_epochs(num_epochs)",
    setup=f"num_epochs={runtime}",
    number=1,
    globals=globals())

print(f"learning rate = {LR}")
print(f"momentum = {MO}")

## Test Data

In [ ]:
ins_dataset_prim = Tomography(
    df= prim_data,
    transform= transforms.Compose([
        transforms.Resize(20),
        transforms.CenterCrop(20),
        transforms.ToTensor(),
        transforms.Normalize(prim_mean, prim_std),
    ]),
)

primlab_loader = torch.utils.data.DataLoader(
    ins_dataset_prim,
    batch_size=batch_size, # Forward pass only so batch size can be larger
    shuffle=False,
    num_workers=0,
    pin_memory=True)

counter = []

for numb in range(len(prim_df['path'])):
  image, label = primlab_loader.dataset[numb]
  image = image.type(torch.cuda.FloatTensor)
  classe = model_gpu.forward(image.unsqueeze(0))
  _ , predicted = torch.max(classe, 1)
  counter.append(predicted.item())
  print(f'Label: {label} , Class: {predicted.item()}')

In [ ]:
print(f'The number of predictions is:{len(counter)}')
prim_names = [
  "Meas194",
  "Meas196",
  "Meas208",
  "Meas209",
  "Meas214",
  "Meas218",
  "Meas226",
  "Meas227"
]

In [ ]:
meas_194 = counter[0:250]
meas_196 = counter[250:500]
meas_208 = counter[500:750]
meas_209 = counter[750:1000]
meas_214 = counter[1000:1250]
meas_218 = counter[1250:1500]
meas_226 = counter[1500:1750]
meas_227 = counter[1750:2000]

In [ ]:
plt.hist(meas_194, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_194')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_196, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_196')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_208, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_208')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_209, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_209')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_214, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_214')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_218, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_218')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_226, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_226')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
plt.hist(meas_227, bins = [0, 1, 2, 3, 4, 5], rwidth = 0.9, align = 'left');
plt.title('Meas_227')
plt.xlabel('Class Prediction');
plt.ylabel('Count');
plt.xticks(np.arange(5), ('Bubble', 'Plug', 'Stratified', 'Slug', 'Transient'))
plt.show()

In [ ]:
# Save model
#path = f"/content/gdrive/My Drive/Models/{layers}_Layers_40avgims.py"
#torch.save(model_gpu, F"{path}")

### Vis

In [ ]:
from PIL import Image
from matplotlib import cm
PCA_List = []
for i in range(len(data_df)):
  PCA_List.append(np.array(Image.open(data_df.path[i])))
  if i%100 == 0:
    print(f"{(i/len(data_df)*100):.2f}% complete")

In [ ]:
x = len(PCA_List)
y = len(PCA_List[1].flatten())
z = np.zeros((x,y))

for i in range(x):
  z[i] = PCA_List[i].flatten()
pca = PCA(10)
z_norm = normalize(z, norm='l1', axis=1, copy=True, return_norm=False)
lower_dimension_data = pca.fit_transform(z_norm)
approx = pca.inverse_transform(lower_dimension_data)

In [ ]:
pca.components_[1]
loading_scores = pd.Series(pca.components_[1])
loading_scores.plot.bar(y='Loading Score')

In [ ]:
plt.scatter(lower_dimension_data[:, 0], lower_dimension_data[:, 1],
            c=data_df["class"],edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))

plt.title("Variance Explained: 0.874")
plt.xlabel('component 1')
plt.ylabel('component 2')
cbar = plt.colorbar(ticks = [0,1,2,3,4])
cbar.ax.set_yticklabels(['Bubble', 'Plug','Stratified','Slug', 'Annular'])
# CLASSES ARE AS FOLLOWS:
# Bubble = 0
# Annular = 1
# Plug = 2
# Stratified = 3
# Slug = 4
#or
#  "Bubble_Flow_Wang",
#  "Plug_Flow",
#  "Stratified_Flow_balancedgroup",
#  "Slug_Flow_Wang",
#  "Transient_Flow_balancedgroup"


In [ ]:
plt.scatter(lower_dimension_data[:, 0],
            lower_dimension_data[:, 2],
            c=data_df["class"],
            edgecolor='none',
            alpha=0.75,
            cmap=plt.cm.get_cmap('Set3', 5))


# zip joins x and y coordinates in pairs
#for x,y in zip(lower_dimension_data[:, 0], lower_dimension_data[:, 1]):
#    label = "{:1f}".format()
#    plt.annotate(label, # this is the text
#                 (x,y), # this is the point to label
#                 textcoords="offset points", # how to position the text
#                 xytext=(0,10), # distance from text to points (x,y)
#                 ha='center') # horizontal alignment can be left, right or center
plt.title("Variance Explained: 0.802")
plt.xlabel('component 1')
plt.ylabel('component 3')
cbar = plt.colorbar(ticks = [0,1,2,3,4])
cbar.ax.set_yticklabels(['Bubble', 'Plug','Stratified','Slug', 'Annular'])

# CLASSES ARE AS FOLLOWS:
# Bubble = 0
# Annular = 1
# Plug = 2
# Stratified = 3
# Slug = 4


In [ ]:
plt.scatter(lower_dimension_data[:, 0], lower_dimension_data[:, 3],
            c=data_df["class"],edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))
# zip joins x and y coordinates in pairs
#for x,y in zip(lower_dimension_data[:, 0], lower_dimension_data[:, 1]):
#    label = "{:1f}".format()
#    plt.annotate(label, # this is the text
#                 (x,y), # this is the point to label
#                 textcoords="offset points", # how to position the text
#                 xytext=(0,10), # distance from text to points (x,y)
#                 ha='center') # horizontal alignment can be left, right or center
plt.title("Variance Explained: 0.776")
plt.xlabel('component 1')
plt.ylabel('component 4')
cbar = plt.colorbar(ticks = [0,1,2,3,4])
cbar.ax.set_yticklabels(['Bubble', 'Plug','Stratified','Slug', 'Annular'])

# CLASSES ARE AS FOLLOWS:
# Bubble = 0
# Annular = 1
# Plug = 2
# Stratified = 3
# Slug = 4


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

x =lower_dimension_data[:, 0]
y =lower_dimension_data[:, 1]
z =lower_dimension_data[:, 2]

ax.scatter (x, y,  z,
            marker='o',
            c=data_df["class"],
            alpha=0.75,
            cmap=plt.cm.get_cmap('Set3', 5))

ax.set_xlabel('component 1')
ax.set_ylabel('component 2')
ax.set_zlabel('component 3')

plt.show()

In [ ]:
pca.components_.shape

In [ ]:
imshow(PCA_List[2].reshape(20,20), cmap = "gray")

In [ ]:
imshow(approx[2].reshape(20,20), cmap = "gray")

In [ ]:
Image.open(data_df.path[2])

In [ ]:
print("znorm",z_norm.shape)
print("Components Shape",pca.components_.shape)
print("PCA Components",pca.components_)
print("Variance Shape",pca.explained_variance_.shape)
print("PCA Variances",pca.explained_variance_)

print("znorm Shape",z_norm.shape)
print("Lower dim Shape",lower_dimension_data.shape)

In [ ]:
v = pca.explained_variance_ratio_.cumsum()
v2 = pca.explained_variance_ratio_
v[:]
plt.bar(x = range(1,len(v2)+1), height = v2, tick_label = range(1,len(v2)+1))
plt.ylabel('Percentage of variance explained')
plt.xlabel('Principal component')
plt.title('Secondary labels averaged every 40th frame')
pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[3] 
approx[1].reshape(20,20).shape
PCA_List[1].shape
PCAed_List = []

for i in range(len(approx)):
  PCAed_List.append(approx[i].reshape(20,20))
for i in range(len(PCAed_List)):
  PCAed_List[i] = torch.from_numpy(PCAed_List[i]).unsqueeze(0).type(torch.FloatTensor)
data_df["PCA_ims"] = PCAed_List

### PCA application




In [ ]:
class Tomography(Dataset):

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        x = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['class'][index]))

        if self.transform:
            x = self.transform(x)

        return x, y 
#Class for PCAed images only
class Tomography_pca(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        z = self.df['PCA_ims'][index]
        y = torch.tensor(int(self.df['class'][index]))
        return z, y #,z/x

In [ ]:
train_split = 0.70 # Defines the ratio of train/valid/test data.
valid_split = 0.10
batch_size = 32

train_size = int(len(data_df)*train_split)
valid_size = int(len(data_df)*valid_split)

ins_dataset_train = Tomography_pca(
    df=data_df[:train_size],
    transform=data_transform,
)

ins_dataset_valid = Tomography_pca(
    df=data_df[train_size:(train_size + valid_size)].reset_index(drop=True),
    transform=data_transform,
)

ins_dataset_test = Tomography_pca(
    df=data_df[(train_size + valid_size):].reset_index(drop=True),
    transform=data_transform,
)

train_loader = torch.utils.data.DataLoader(
    ins_dataset_train,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory = True

)
valid_loader = torch.utils.data.DataLoader(
    ins_dataset_valid,
    batch_size=batch_size, # Forward pass only so batch size can be larger
    shuffle=False,
    num_workers=0,
    pin_memory = True)
test_loader = torch.utils.data.DataLoader(
    ins_dataset_test,
    batch_size=batch_size, # Forward pass only so batch size can be larger
    shuffle=False,
    num_workers=0,
    pin_memory = True)

# Architecture

In [ ]:
# Convolutional neural network
names = [0,1,2,3,4]
class ConvNet(nn.Module):
    def __init__(self,
                 num_classes=len(names)):
        super(ConvNet, self).__init__()
  
        # Add network layers here
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #,nn.Dropout(p = 0.3))
        #self.conv2 = nn.Sequential(
            #nn.Conv2d(16, 24, 4),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2, stride=2),
            #nn.Dropout(p = 0.3))
        #self.fc1 = nn.Linear(216, 512) 
        #self.fc2 = nn.Linear(512, num_classes)
        ## With just one Conv
        self.fc1 = nn.Linear(1296, num_classes) 
        #self.fc2 = nn.Linear(216, num_classes) 

    def forward(self, x):

        # Complete the graph
        out = self.conv1(x)
        #out = self.conv2(out)
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        #out = self.fc2(out)
        
        return out
    
model = ConvNet()
layers = 2
runtime = 150

In [ ]:
#train_model_epochs(num_epochs)
from torch import optim
LR = 0.025
MO = 0.9
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_gpu = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
# Stochastic gradient descent
optimizer = optim.SGD(model_gpu.parameters(), lr=LR, momentum=MO) 

model_gpu = model_gpu.to(device)
cpu_train_time = timeit.timeit(
    "train_cnn_epochs(num_epochs)",
    setup=f"num_epochs={runtime}",
    number=1,
    globals=globals(),
)

print(f"learning rate = {LR}")
print(f"momentum = {MO}")
test_stats(model_gpu)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

labeltot, predtot, _, _ = test_stats(model_gpu)
cm = confusion_matrix(labeltot, predtot)
plot_confusion_matrix(cm, names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(cm.diagonal())/sum(sum(cm[:,:]))):.2f}% \n"
              )
test_stats(model_gpu)

### PCA dataloaders

In [ ]:
len(noshuf_df['class'])

In [ ]:
bublist = []
pluglist = []
stratlist = []
sluglist = []
tranlist = []
for i in range(len(noshuf_df)):
  if noshuf_df['class'][i] == 0:
    bublist.append(np.array(Image.open(noshuf_df.path[i])).flatten())
  elif noshuf_df['class'][i] == 1:
    pluglist.append(np.array(Image.open(noshuf_df.path[i])).flatten())
  elif noshuf_df['class'][i] == 2:
    stratlist.append(np.array(Image.open(noshuf_df.path[i])).flatten())
  elif noshuf_df['class'][i] == 3:
    sluglist.append(np.array(Image.open(noshuf_df.path[i])).flatten())
  elif noshuf_df['class'][i] == 4:
    tranlist.append(np.array(Image.open(noshuf_df.path[i])).flatten())

In [ ]:
len(tranlist)

In [ ]:
imshow(bublist[20].reshape(20,20), cmap = 'gray')

In [ ]:
pca1 = PCA(2)
pca2 = PCA(2)
pca3 = PCA(2)
pca4 = PCA(2)
pca5 = PCA(2)
bub_norm = normalize(bublist, norm='l1', axis=1, copy=True, return_norm=False)
plug_norm = normalize(pluglist, norm='l1', axis=1, copy=True, return_norm=False)
strat_norm = normalize(stratlist, norm='l1', axis=1, copy=True, return_norm=False)
slug_norm = normalize(sluglist, norm='l1', axis=1, copy=True, return_norm=False)
trans_norm = normalize(tranlist, norm='l1', axis=1, copy=True, return_norm=False)

lower_dimension_bub = pca1.fit_transform(bub_norm)
bub_approx = pca1.inverse_transform(lower_dimension_bub)
lower_dimension_plug = pca2.fit_transform(plug_norm)
plug_approx = pca2.inverse_transform(lower_dimension_plug)
lower_dimension_strat = pca3.fit_transform(strat_norm)
strat_approx = pca3.inverse_transform(lower_dimension_strat)
lower_dimension_slug = pca4.fit_transform(slug_norm)
slug_approx = pca4.inverse_transform(lower_dimension_slug)
lower_dimension_trans = pca5.fit_transform(trans_norm)
trans_approx = pca5.inverse_transform(lower_dimension_trans)



In [ ]:
pca2.components_.shape

In [ ]:
print(pca1.explained_variance_ratio_.cumsum(),
      pca2.explained_variance_ratio_.cumsum(),
      pca3.explained_variance_ratio_.cumsum(),
      pca4.explained_variance_ratio_.cumsum(),
      pca5.explained_variance_ratio_.cumsum())

In [ ]:
plt.scatter(lower_dimension_bub[:, 0], lower_dimension_bub[:, 1],
          edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))
plt.title("Bubble Flow (secondary): 0.994 variance")
plt.xlabel('component 1')
plt.ylabel('component 2')

In [ ]:
plt.scatter(lower_dimension_plug[:, 0], lower_dimension_plug[:, 1],
          edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))
plt.title("Plug Flow: 0.753 variance")
plt.xlabel('component 1')
plt.ylabel('component 2')

In [ ]:
plt.scatter(lower_dimension_strat[:, 0], lower_dimension_strat[:, 1],
          edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))
plt.title("Stratified Flow: 0.716 variance")
plt.xlabel('component 1')
plt.ylabel('component 2')

In [ ]:
plt.scatter(lower_dimension_slug[:, 0], lower_dimension_slug[:, 1],
          edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))
plt.title("Slug Flow (secondary): 0.808 variance")
plt.xlabel('component 1')
plt.ylabel('component 2')

In [ ]:
plt.scatter(lower_dimension_trans[:, 0], lower_dimension_trans[:, 1],
          edgecolor='none', alpha=0.75, cmap=plt.cm.get_cmap('Set3', 5))
plt.title("Transient Flow: 0.767 variance")
plt.xlabel('component 1')
plt.ylabel('component 2')

bub_approx.shape
imshow(stratlist[12].reshape(20,20), cmap = 'gray')
imshow(strat_approx[12].reshape(20,20), cmap = 'gray')
lst = []
for i in range(len(bub_approx)):
  lst.append(bub_approx[i].reshape(20,20))
for i in range(len(plug_approx)):
  lst.append(plug_approx[i].reshape(20,20))
for i in range(len(strat_approx)):
  lst.append(strat_approx[i].reshape(20,20))
for i in range(len(slug_approx)):
  lst.append(slug_approx[i].reshape(20,20))
for i in range(len(trans_approx)):
  lst.append(trans_approx[i].reshape(20,20))
for i in range(len(lst)):
  lst[i] = torch.from_numpy(lst[i]).unsqueeze(0).type(torch.FloatTensor)
len(lst)
noshuf_df
noshuf_df["PCA_ims"] = lst
noshuf_df = noshuf_df.sample(frac=1).reset_index(drop=True) # Shuffles the data
noshuf_df

In [ ]:
#Class for PCAed images only

class Tomography(Dataset):
    """ Tomography dataset. """

    def __init__(self, df, transform=None):
        """
        Args:
            image_dir (string): Directory with all the images
            df (DataFrame object): Dataframe containing the images, paths and classes
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load image from path and get label
        z = self.df['PCA_ims'][index]
        y = torch.tensor(int(self.df['class'][index]))

        return z, y #,z/x

In [ ]:
imshow(lst[597].reshape(20,20), cmap = "gray")

In [ ]:
train_split = 0.70 # Defines the ratio of train/valid/test data.
valid_split = 0.10
batch_size = 32

train_size = int(len(noshuf_df)*train_split)
valid_size = int(len(noshuf_df)*valid_split)

ins_dataset_train = Tomography(
    df=noshuf_df[:train_size],
    transform=data_transform,
)

ins_dataset_valid = Tomography(
    df=noshuf_df[train_size:(train_size + valid_size)].reset_index(drop=True),
    transform=data_transform,
)

ins_dataset_test = Tomography(
    df=noshuf_df[(train_size + valid_size):].reset_index(drop=True),
    transform=data_transform,
)
train_loader = torch.utils.data.DataLoader(
    ins_dataset_train,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory = True

)
valid_loader = torch.utils.data.DataLoader(
    ins_dataset_valid,
    batch_size=batch_size, # Forward pass only so batch size can be larger
    shuffle=False,
    num_workers=0,
    pin_memory = True
)
test_loader = torch.utils.data.DataLoader(
    ins_dataset_test,
    batch_size=batch_size, # Forward pass only so batch size can be larger
    shuffle=False,
    num_workers=0,
    pin_memory = True
)

### 2 Layer w PCA


In [ ]:
# Convolutional neural network
class ConvNet(nn.Module):
    
    def __init__(self, num_classes=len(names)):
        super(ConvNet, self).__init__()
  
        # Add network layers here
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels= 1,
                      out_channels=16,
                      kernel_size= (3,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2))
            #,nn.Dropout(p = 0.3))
        #self.conv2 = nn.Sequential(
            #nn.Conv2d(16, 24, 4),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2, stride=2),
            #nn.Dropout(p = 0.3))
        #self.fc1 = nn.Linear(216, 512) 
        #self.fc2 = nn.Linear(512, num_classes)
        ## With just one Conv
        self.fc1 = nn.Linear(1296, num_classes) 
        #self.fc1 = nn.Linear(216, num_classes) 

    def forward(self, x):

        # Complete the graph
        out = self.conv1(x)
        #out = self.conv2(out)
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        #out = self.fc2(out)
        
        return out
    
model = ConvNet()
layers = 2
runtime = 50

# Initialization CNN

In [ ]:
#train_model_epochs(num_epochs)
from torch import optim
import timeit
LR = 0.02
MO = 0.9
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_gpu = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
# Stochastic gradient descent
optimizer = optim.SGD(model_gpu.parameters(),
                      lr=LR,
                      momentum=MO) 
model_gpu = model_gpu.to(device)
cpu_train_time = timeit.timeit(
    "train_cnn_epochs(num_epochs)",
    setup=f"num_epochs={runtime}",
    number=1,
    globals=globals(),
)
print(f"learning rate = {LR}")
print(f"momentum = {MO}")
test_stats(model_gpu)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

labeltot, predtot, _, _ = test_stats(model_gpu)
cm = confusion_matrix(labeltot, predtot)

In [ ]:
plot_confusion_matrix(cm, names)
print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(cm.diagonal())/sum(sum(cm[:,:]))):.2f}% \n"
              )
test_stats(model_gpu)


In [ ]:
20/180 * () + 60/180 *() + 10/180 *()

# Decision tree


In [ ]:
label_set = 1
grp_set = 5 # 6, 7
lbl = ('Primary_Labels','Secondary_Labels')

grp_list = ('Vectors_Single40th',  'Vectors_AV10', 'Vectors_AV20', 'Vectors_AV30',
            'Vectors_AV40', 'Vectors_STD', 'Vectors_ACF1', 'Vectors_ACF5')

directory = f"/content/gdrive/My Drive/Tomography_Images/{lbl[label_set]}/{grp_list[grp_set]}/"

confusion_matrix_title = ['single image', '10 frames average', '20 frames average',
                          '30 frames average', '40 frames average',
                          '40 frames standard deviation', 'Auto-correlation: Lag 1', 'Auto-correlation: Lag 5']

if label_set == 0:
  label, names, name = 'Primary Labels', ["Bubble_flow","Plug_flow","Stratified_flow","Slug_flow","Annular_flow"], ["Bubble","Plug","Stratified","Slug","Annular"]
elif label_set == 1:
  label, names, name = 'Secondary Labels', ["Bubble_flow","Plug_flow","Stratified_flow","Slug_flow","Transient_flow"], ["Bubble","Plug","Stratified","Slug","Transient"]

all_experiments = ["194","196","198","200","202", "204", "206", "208", "209","210",
                   "212","214","216","218","220","221","222","223","224","225","226",
                   "227","228"]

In [ ]:
# Benign images we will assign class 0, and malignant as 1
paths, classes = get_data(directory, names)
SVM_data = {
    'path': paths,
    'class': classes
    }
SVM_df = pd.DataFrame(SVM_data, columns=['path', 'class'])
SVM_df = SVM_df.sample(frac=1).reset_index(drop=True) # Shuffles the data
SVM_df['path'][1]
# Code for both images
lst = []
for i in tqdm(range(len(SVM_df)), desc='Loading data...'):
  test = pd.read_csv(SVM_df['path'][i])

  if grp_list[grp_set] in ['Vectors_ACF1','Vectors_ACF5']:
    t1 = t1.T.reset_index(drop=True).T.fillna(0)
    t1.insert(316, 'class', SVM_df['class'][i])
    lst.append(t1)

  if grp_list[grp_set] not in  ['Vectors_ACF1' ,'Vectors_ACF5']:
    t1 = test.iloc[:, 0:316]   #  Taking first images
    t2 = test.iloc[:,316:634]  #  and second images
    t1 = t1.T.reset_index(drop=True).T
    t2 = t2.T.reset_index(drop=True).T
    t1.insert(316, 'class', SVM_df['class'][i])   #  Inserting Class for both imgs
    t2.insert(316, 'class', SVM_df['class'][i])
    lst.append(t1)
    lst.append(t2)

tot = pd.concat(lst).to_numpy() # 
tot # (instances x parameters)
# Code for first images only
lst = []
for i in range(len(SVM_df)):
  test = pd.read_csv(SVM_df['path'][i])
  test.insert(316, 'class', SVM_df['class'][i])
  test2 = test.iloc[:, 0:317]
  lst.append(test2)
tot = pd.concat(lst).to_numpy()
tot = np.nan_to_num(tot,nan=0.0)
labels = tot[:,-1]
labels = [int(class_tag) for class_tag in labels]
images = tot[:,0:-1]
unique(labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images,labels)
y_train = [int(target) for target in y_train]
y_test = [int(target) for target in y_test]

print(f'Training data and target sizes: \n{len(X_train)}, {len(y_train)}')
print(f'Test data and target sizes: \n{len(X_test)}, {len(y_test)}')
X_test.shape
print('\n')
X_test.shape
print('\n')
X_train
print('\n')
X_train.shape

## Data augmentation: class enrichment


In [ ]:
df = pd.DataFrame(zip(X_train,y_train)).rename(columns={0: "path"}).rename(columns={1: "class"})
x = unique(df['class'])
size = []
extend, boundary = {}, {}
for i in x:
  porp = listcheck(df['class'],i)
  if i == 0:
    lst0 = df.sort_values("class").iloc[0:porp]
    l0 = len(lst0)
  if i == 1:
    lst1 = df.sort_values("class").iloc[l0:l0+porp]
    l1 = len(lst1)
  if i == 2:
    lst2 = df.sort_values("class").iloc[l0+l1:l0+l1+porp]
    l2 = len(lst2)
  if i == 3:
    lst3 = df.sort_values("class").iloc[l0+l1+l2:l0+l1+l2+porp]
    l3 = len(lst3)
  if i == 4:
    lst4 = df.sort_values("class").iloc[l0+l1+l2+l3:l0+l1+l2+l3+porp]
  print(f"class {i} represents {listcheck(df['class'],i)} of the training elements")
  size.append(listcheck(df['class'],i))

for i in x:
  diff = max(size) - listcheck(df['class'],i)
  extend[i] = diff
  print(f"{diff} difference for {i}")
  for j in range(diff):
    pass

tba0 = lst0.sample(extend[0], replace = True)
tba1 = lst1.sample(extend[1], replace = True)
tba2 = lst2.sample(extend[2], replace = True)
tba3 = lst3.sample(extend[3], replace = True)
tba4 = lst4.sample(extend[4], replace = True)
tba1.iloc[1][0].shape

In [ ]:
imag0 = []
for i in tqdm(range(len(tba0)), desc='Adding Gaussian noise....'):
  test = tba0.iloc[i][0]
  for j in range(316):
    test[j] += abs(test[j] + np.random.normal(0,0.03,1))
  imag0.append(test)
imag1 = []
for i in tqdm(range(len(tba1)), desc='Adding Gaussian noise....'):
  test = tba1.iloc[i][0]
  for j in range(316):
    test[j] += abs(test[j] + np.random.normal(0,0.03,1))
  imag1.append(test)
imag2 = []
for i in tqdm(range(len(tba2)), desc='Adding Gaussian noise....'):
  test = tba2.iloc[i][0]
  for j in range(316):
    test[j] += abs(test[j] + np.random.normal(0,0.03,1))
  imag2.append(test)

imag3 = []
for i in tqdm(range(len(tba3)), desc='Adding Gaussian noise....'):
  test = tba3.iloc[i][0]
  for j in range(316):
    test[j] += abs(test[j] + np.random.normal(0,0.03,1))
  imag3.append(test)

imag4 = []
for i in tqdm(range(len(tba4)), desc='Adding Gaussian noise....'):
  test = tba4.iloc[i][0]
  for j in range(316):
    test[j] += abs(test[j] + np.random.normal(0,0.03,1))
  imag4.append(test)
zeros = [0]*len(imag0)
ones = [1]*len(imag1)
twos = [2]*len(imag2)
threes = [3]*len(imag3)
fours = [4]*len(imag4)

In [ ]:
x = df['path']
y = df['class']
zip0 = list(zip(imag0,zeros))
class_zero = pd.DataFrame(zip0, columns=['path','class'])
zip1 = list(zip(imag1,ones))
class_one = pd.DataFrame(zip1, columns=['path','class'])
zip2 = list(zip(imag2,twos))
class_two = pd.DataFrame(zip2, columns=['path','class'])
zip3 = list(zip(imag3,threes))
class_three = pd.DataFrame(zip3, columns=['path','class'])
zip4 = list(zip(imag4,fours))
class_four = pd.DataFrame(zip4, columns=['path','class'])
zip10 = list(zip(x,y))
original_train = pd.DataFrame(zip10, columns=['path','class'])

balanced_train = pd.concat([class_zero, class_one, class_two, class_three, class_four, original_train],
                           ignore_index = True)

freq_train = [listcheck(balanced_train['class'], 0),
              listcheck(balanced_train['class'], 1),
              listcheck(balanced_train['class'], 2),
              listcheck(balanced_train['class'], 3),
              listcheck(balanced_train['class'], 4)]

classestrain = np.arange(5, dtype=int)
plt.bar(classestrain, freq_train)
plt.title('Secondary label distribution')
plt.xlabel('Flow Type')
plt.ylabel('Frequency')
plt.xticks(classestrain, name)
plt.show()


# **Balanced vs unbalanced training routine selection**

In [ ]:
from numpy import asarray
###########################################
balanced_training = True # <--- alter this
###########################################
# training data for balanced set
y_train_B = asarray(balanced_train['class'].values.tolist())
X_train_B = asarray(balanced_train['path'].values.tolist())
# want int targets
y_train = [int(target) for target in y_train]

if balanced_training == True:
  X_train = X_train_B
  y_train = y_train_B
  print('Using balanced training set')

## For PCA Data Use

In [ ]:
img_norm = normalize(images)
pca = PCA(.999)
lower_dimension_data = pca.fit_transform(img_norm)
approx = pca.inverse_transform(lower_dimension_data)

In [ ]:
v = pca.explained_variance_ratio_.cumsum()
#plt.plot(v)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(approx,labels)
print(f'Training data and target sizes: \n{len(X_train)}, {len(y_train)}')
print(f'Test data and target sizes: \n{len(X_test)}, {len(y_test)}')

#  Decision Tree: Grid search

In [ ]:
dec_tree = DecisionTreeClassifier()
n_leaf = np.linspace(50, 150, 50).astype(int) # 52, ..., 148, 150
# hyperparameter grid-search space
params = {'max_leaf_nodes': n_leaf,
          'min_samples_split': [2, 3, 4],
          'criterion': ['gini', 'entropy']}

classifier = GridSearchCV(estimator=dec_tree,
                          param_grid=params,
                          n_jobs=-1,
                          verbose=10,
                          cv=5)
start = time()
classifier.fit(X_train,
               y_train)

best_params = classifier.best_params_
print(f"GridSearchCV took {(time() - start):.2f} seconds")
best = DecisionTreeClassifier(**best_params)
best.fit(X_train,
         y_train)
y_pred = best.predict(X_test)
results = confusion_matrix(y_test,
                           y_pred)

plot_confusion_matrix(results,
                      [m.replace('_', ' ') for m in names],
                      title =f'Grid search decision tree confusion matrix:\n{confusion_matrix_title[grp_set]}',
                      cmap=plt.cm.Blues)

print(classification_report(y_test,
                            y_pred,
                            target_names=[m.replace('_', ' ') for m in names]))

# Decision tree: Random Search


In [ ]:
n_iter = 20 # number of randomized samples of the hyperparameter space -----> quick 
classifier = RandomizedSearchCV(dec_tree,
                                param_distributions=params,
                                n_iter=n_iter,
                                cv=5,
                                n_jobs=-1,
                                verbose=10)
start = time()
classifier.fit(X_train,
               y_train)

print(f"RandomizedSearchCV took {(time() - start):.2f} seconds")
best_params = classifier.best_params_
best = DecisionTreeClassifier(**best_params)

best.fit(X_train,
         y_train)

y_pred = best.predict(X_test)
results = confusion_matrix(y_test,
                          y_pred)

plot_confusion_matrix(results,
                      [m.replace('_', ' ') for m in names],
                      title =f'Randomized search decision tree confusion matrix:\n{confusion_matrix_title[grp_set]}',
                      cmap=plt.cm.Blues)

print(classification_report(y_test,
                            y_pred,
                            target_names=[m.replace('_', ' ') for m in names]))

# Decision tree: Bayes optimization

In [ ]:
dec_tree = DecisionTreeClassifier()
n_leaf = np.linspace(50, 150, 50).astype(int) # 52, ..., 148, 150
n_iter = 20
# hyperparameter grid-search space
params = {'max_leaf_nodes': n_leaf,
          'min_samples_split': [2, 3, 4],
          'criterion': ['gini', 'entropy']}
classifier = BayesSearchCV(dec_tree,
                           search_spaces=params,
                           n_iter=n_iter,
                           cv=5,
                           verbose=1)
start = time()
X_train = X_train.astype(float)
classifier.fit(X_train,
               y_train)

print(f"BayesSearchCV took {(time() - start):.2f} seconds")

best_params = classifier.best_params_
best = DecisionTreeClassifier(**best_params)

best.fit(X_train,
         y_train)

y_pred = best.predict(X_test)
results = confusion_matrix(y_test,
                          y_pred)

plot_confusion_matrix(results,
                      [m.replace('_', ' ') for m in names],
                      title =f'Bayesian search decision tree confusion matrix:\n{confusion_matrix_title[grp_set]}',
                      cmap=plt.cm.Blues)

print(classification_report(y_test,
                            y_pred,
                            target_names=[m.replace('_', ' ') for m in names]))

# plt.figure(figsize=(150,50))
# plot_tree(best,
#          class_names=[m.replace('_', ' ') for m in names],
#          filled = True,
#          fontsize=18,
#          rounded =True)

# ETR_var_heatmap = np.reshape(best.feature_importances_, (20,20))
# plt.imshow(ETR_var_heatmap)

# Random Forest: Grid-search

In [ ]:
rf = RandomForestClassifier()
# Hyperparameter space
params = {'max_leaf_nodes': np.linspace(20, 150, 10).astype(int),
          'criterion': ['gini', 'entropy'],
          'n_estimators': np.linspace(50, 150, 10).astype(int) }
n_iter = 20 # number of randomized samples 
classifier = GridSearchCV(estimator=rf,
                          param_grid=params,
                          n_jobs=5,
                          verbose = 10)
start = time()
classifier.fit(X_train,
               y_train)
print(f"GridSearchCV took {(time() - start):.2f} seconds" )

best_params = classifier.best_params_
best = RandomForestClassifier(**best_params)
best.fit(X_train,
         y_train)
y_pred = best.predict(X_test)
results = confusion_matrix(y_test,
                           y_pred)

plot_confusion_matrix(results,
                      [m.replace('_', ' ') for m in names],
                      title =f'Grid search random forest confusion matrix:\n{confusion_matrix_title[grp_set]}',
                      cmap=plt.cm.Blues)

print(classification_report(y_test,
                            y_pred,
                            target_names=[m.replace('_', ' ') for m in names]))

# Random forest : Randomized search


In [ ]:
n_iter = 20 # number of randomized hyperparameter space sampling 
classifier = RandomizedSearchCV(rf,
                                param_distributions=params,
                                n_iter=n_iter)
start = time()
classifier.fit(X_train,
               y_train)

print(f"RandomizedSearchCV took {(time() - start):.2f} seconds" )

best_params = classifier.best_params_
best = RandomForestClassifier(**best_params)
best.fit(X_train,
         y_train)

y_pred = best.predict(X_test)
results = confusion_matrix(y_test,
                           y_pred)

plot_confusion_matrix(results,
                      [m.replace('_', ' ') for m in names],
                      title =f'Randomized search random forest confusion matrix:\n{confusion_matrix_title[grp_set]}',
                      cmap=plt.cm.Blues)

print(classification_report(y_test,
                      y_pred,
                      target_names=[m.replace('_', ' ') for m in names]))

# Random forest: Bayes Optimization

In [ ]:
rf = RandomForestClassifier()
# Hyperparameter space
params = {'max_leaf_nodes': np.linspace(20, 150, 10).astype(int),
          'criterion': ['gini', 'entropy'],
          'n_estimators': np.linspace(50, 150, 10).astype(int) }
n_iter = 20 # number of randomized samples 
classifier = BayesSearchCV(rf,
                           search_spaces=params,
                           n_iter=n_iter,
                           cv=5,
                           verbose=1)
start = time()
classifier.fit(X_train,
               y_train)
print(f"BayesSearchCV took {(time() - start):.2f} seconds")

best_params = classifier.best_params_
best = RandomForestClassifier(**best_params)
best.fit(X_train,
         y_train)
y_pred = best.predict(X_test)
results = confusion_matrix(y_test,
                          y_pred)
plot_confusion_matrix(results,
                      [m.replace('_', ' ') for m in names],
                      title =f'Bayesian search random forest confusion matrix:\n{confusion_matrix_title[grp_set]} balanced training set',
                      cmap=plt.cm.Blues)

print(classification_report(y_test,
                      y_pred,
                      target_names=[m.replace('_', ' ') for m in names]))

In [ ]:
rf = classifier.best_estimator_
classifier.optimizer_results_[0]
_ = plot_objective(classifier.optimizer_results_[0])
plt.show()
rf
print(f"val. score: {rf.best_score_}" )
print(f"test score: {rf.score(X_test, y_test)}")
rf_pred = rf.predict(X_test)
rf_cm = metrics.confusion_matrix(y_test, rf_pred)
plot_confusion_matrix(rf_cm, names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(rf_cm.diagonal())/sum(sum(rf_cm[:,:]))):.2f}% \n"
              )
#rf.estimators_[0]
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
importances = (importances/max(importances))
error_rate = []
for i in range(50, 100):
    rf.set_params(n_estimators=i)
    rf.fit(X_train, y_train)
    oob_error = 1 - rf.oob_score_
    error_rate.append((i, oob_error))
error_rate[0]
x,y = [],[]
for i in range(len(error_rate)):
  x.append(error_rate[i][0])
  y.append(error_rate[i][1])
print(y)
plt.plot(x,y, label = 'RandomForest')
plt.xlim(50, 60)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()


# **Additional models: not reported**

In [ ]:
import matplotlib.pyplot as plt

from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

print(__doc__)

RANDOM_STATE = 123

# Generate a binary classification dataset.
X, y = make_classification(n_samples=500, n_features=25,
                           n_clusters_per_class=1, n_informative=15,
                           random_state=RANDOM_STATE)


ensemble_clfs = [
    ("RandomForestClassifier,ax_features m='sqrt'",
        RandomForestClassifier(warm_start=True, oob_score=True,
                               max_features="sqrt",
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features='log2'",
        RandomForestClassifier(warm_start=True, max_features='log2',
                               oob_score=True,
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features=None",
        RandomForestClassifier(warm_start=True, max_features=None,
                               oob_score=True,
                               random_state=RANDOM_STATE))
]

# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

# Range of `n_estimators` values to explore.
min_estimators = 15
max_estimators = 175

for label, clf in ensemble_clfs:
    for i in range(min_estimators, max_estimators + 1):
        clf.set_params(n_estimators=i)
        clf.fit(X, y)

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    print(ys)
    #plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

In [ ]:
for err in error_rate:
    x, y = err
    plt.plot(x, y)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = []

# Range of `n_estimators` values to explore.
min_estimators = 15
max_estimators = 175

for i in range(min_estimators, max_estimators + 1):
    rf.set_params(n_estimators=i)
    rf.fit(X_train, y_train)

    # Record the OOB error for each `n_estimators=i` setting.
    oob_error = 1 - rf.oob_score_
    error_rate.append((i, oob_error))

# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

In [ ]:
importances

In [ ]:
# Saving NumPy array as a csv file
array_rain_fall = np.loadtxt(fname="rain-fall.csv",
                             delimiter=",")
np.savetxt(fname="saved-rain-fall-row-col-names.csv",
           delimiter=",",
           X=array_rain_fall)

# Check generated csv file after loading it

array_rain_fall_csv_saved = np.loadtxt(
    fname="saved-rain-fall-row-col-names.csv", delimiter=","
)

print("NumPy array: \n", array_rain_fall_csv_saved)
print("Shape: ", array_rain_fall_csv_saved.shape)
print("Data Type: ", array_rain_fall_csv_saved.dtype.name)

# We can change our delimeter and save file in tsv or other text format


In [ ]:
# Saving NumPy array as a csv file
array_rain_fall = np.loadtxt(fname="rain-fall.csv", delimiter=",")
np.savetxt(fname="saved-rain-fall-row-col-names.csv", delimiter=",", X=array_rain_fall)

# Check generated csv file after loading it
array_rain_fall_csv_saved = np.loadtxt(
    fname="saved-rain-fall-row-col-names.csv", delimiter=","
)
print("NumPy array: \n", array_rain_fall_csv_saved)
print("Shape: ", array_rain_fall_csv_saved.shape)
print("Data Type: ", array_rain_fall_csv_saved.dtype.name)
# We can change our delimeter and save file in tsv or other text format

In [ ]:
# Saving NumPy array as a csv file
array_rain_fall = np.loadtxt(fname="rain-fall.csv", delimiter=",")
np.savetxt(fname="saved-rain-fall-row-col-names.csv", delimiter=",", X=array_rain_fall)

# Check generated csv file after loading it

array_rain_fall_csv_saved = np.loadtxt(
    fname="saved-rain-fall-row-col-names.csv", delimiter=","
)

print("NumPy array: \n", array_rain_fall_csv_saved)
print("Shape: ", array_rain_fall_csv_saved.shape)
print("Data Type: ", array_rain_fall_csv_saved.dtype.name)

# We can change our delimeter and save file in tsv or other text format


In [ ]:
#tree.plot_tree(rf.estimators_[0], class_names=class_names, filled = True);

## K-fold

In [ ]:
rf_kfold = cross_validate(rf, X_train, y_train, cv=5)
print(rf_kfold['test_score'].mean())

# Adaptive Boosting


In [ ]:
ab = AdaBoostClassifier()
params = { 'n_estimators':np.linspace(20, 150, 16).astype(int) ,
          'learning_rate': np.linspace(0, 2, 200),
          'algorithm':['SAMME', 'SAMME.R']}
ab.fit(X_train, y_train)


## Grid Search


In [ ]:
classifier = GridSearchCV(estimator=ab, param_grid=params, n_jobs=5, verbose = 10)
start = time()
classifier.fit(X_train, y_train)
print("GridSearchCV took %.2f seconds" % (time() - start))

## Random Search


In [ ]:
classifier = RandomizedSearchCV(ab, param_distributions=params, n_iter=n_iter)
start = time()
classifier.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds" % (time() - start)

## Bayes Optimization

In [ ]:
classifier = BayesSearchCV(ab,search_spaces=params,n_iter=n_iter,cv=3)
start = time()
classifier.fit(X_train, y_train)
print("BayesSearchCV took %.2f seconds" % (time() - start))

In [ ]:
ab = classifier.best_estimator_

In [ ]:
_ = plot_objective(classifier.optimizer_results_[0])
plt.show()

In [ ]:
rf_cm = metrics.confusion_matrix(y_test, rf_pred)
plot_confusion_matrix(rf_cm, names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(rf_cm.diagonal())/sum(sum(rf_cm[:,:]))):.2f}% \n"
              )

# SVM

In [ ]:
# Create a classifier: a support vector classifier
svmm = svm.SVC()
Cs = np.linspace(100, 300, 40)

#folds = 5
#skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 999)
classifier = GridSearchCV(estimator=svmm, param_grid=dict(C=Cs), n_jobs=5, cv = 5,
                          return_train_score = True, verbose=10)
classifier.fit(X_train,y_train)
#classifier.fit(X_train,y_train)

In [ ]:
classifier.best_estimator_

In [ ]:
results = classifier.cv_results_
results_df = pd.DataFrame(results)

results_df['split0_test_score']
results_df['param_C']

#best_mean = 
#for i in range(len(results_df)):
#  results_df['mean_test_score '].iloc[i]

results_df

In [ ]:
GridSearch_table_plot(classifier, 'C')

In [ ]:
# Best Classifier for PCA'ed data
classifier = svm.SVC(C = 158.333)
classifier.fit(X_train,y_train)
svm_pred = classifier.predict(X_test)

In [ ]:
# Best Classifier for non-PCA'ed data
classifier = svm.SVC(C = 166.666)
classifier.fit(X_train,y_train)
svm_pred = classifier.predict(X_test)

In [ ]:
print(classification_report(y_test, svm_pred))

In [ ]:
len(X_test[1])

In [ ]:
img = Image.fromarray(X_test[1])

In [ ]:
svm_pred

In [ ]:
y_test

In [ ]:
print("Classification report for classifier %s:\n%s\n" % (classifier, metrics.classification_report(y_test, svm_pred)))

In [ ]:
svm_cm = metrics.confusion_matrix(y_test, svm_pred)
plot_confusion_matrix(svm_cm, class_names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(svm_cm.diagonal())/sum(sum(svm_cm[:,:]))):.2f}% \n"
              )

# KNN


In [ ]:
np.linspace(5,35,7).astype(int)

In [ ]:
leafs = np.linspace(5,35,7).astype(int)
knn = KNeighborsClassifier()
params = { #'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
          'leaf_size': leafs , 
          'metric':['manhattan'],
                    #'minkowski','euclidean','manhattan'],
                     'n_neighbors':[3,5,7,11,13],
                     #'weights':['uniform','distance']
           }
classifier = GridSearchCV(estimator=knn, param_grid=params, n_jobs=5,
                          verbose=10,return_train_score = True)

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
max(results_df['mean_test_score'])

In [ ]:
classifier.best_estimator_

In [ ]:
results = classifier.cv_results_
results_df = pd.DataFrame(results)
results_df

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, metric = 'manhattan')
knn.fit(X_train, y_train)

In [ ]:
knn_pred = knn.predict(X_test)

In [ ]:
knn_cm = metrics.confusion_matrix(y_test, knn_pred)
plot_confusion_matrix(knn_cm, class_names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(knn_cm.diagonal())/sum(sum(knn_cm[:,:]))):.2f}% \n"
              )

## K-fold

In [ ]:
knn_kfold = cross_validate(knn, X_train, y_train, cv=5)
knn_kfold['test_score']

# NN

In [ ]:
dec_tree = DecisionTreeClassifier()
params = {'solver': ['lbfgs'],
          'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ],
          'alpha': 10.0 ** -np.arange(1, 10),
          'hidden_layer_sizes':np.linspace(5, 20,1), 
          'random_state':[0,1,2,3,4,5,6,7,8,9]}

          
classifier = GridSearchCV(estimator=dec_tree, param_grid=params, n_jobs=5)

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
nn.fit(X_train,y_train)

nn_pred = nn.predict(X_test)

In [ ]:
nn_cm = metrics.confusion_matrix(y_test, nn_pred)
plot_confusion_matrix(nn_cm, class_names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(nn_cm.diagonal())/sum(sum(nn_cm[:,:]))):.2f}% \n"
              )

## K-fold

In [ ]:
nn_kfold = cross_validate(nn, X_train, y_train, cv=5)
nn_kfold['test_score']

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

logreg_pred = logreg.predict(X_test)

In [ ]:
logreg_cm = metrics.confusion_matrix(y_test, logreg_pred)
plot_confusion_matrix(logreg_cm, class_names)

print(">>> \n"
              "Test Accuracy: \n"
              f"{(100*sum(logreg_cm.diagonal())/sum(sum(logreg_cm[:,:]))):.2f}% \n"
              )

## K-fold

In [ ]:
logreg_kfold = cross_validate(logreg, X_train, y_train, cv=5)
logreg_kfold['test_score']

In [ ]:
from google.colab import files
uploaded = files.upload()